In [150]:
from skimage import feature
from skimage.io import imread
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import os
# pip install imutils
# pip install opencv-python
import imutils
from itertools import combinations

In [181]:
def get_features(path,file,lgc_return_label = False):
    if lgc_return_label:
        label = file.split("_")[0]
    else:
        label = 'unknown'

    filename = path+file
    img = imread(filename)

    if len(img.shape) != 3:
        a = img[:,:]
        b = img[:,:]
        c = img[:,:]
    else:
        a = img[:,:,0]
        b = img[:,:,1]
        c = img[:,:,2]
    
    ratios = get_ratios(a,b,c)
    correlations = get_correlations(a,b,c)
    max_norm = get_max_norm(a,b,c)
    mean_std = get_mean_std(a,b,c)
    
    parameters = ratios + correlations + max_norm + mean_std 
    parameters = np.asarray(parameters)
    
    
    return parameters,label

In [133]:
def get_ratios(a,b,c):
    vals = []
    for ii,jj in combinations([a,b,c],2):
        vals.append(np.mean(ii)/np.mean(jj))
    return vals

def get_correlations(a,b,c):
    vals = []
    for ii,jj in combinations([a,b,c],2):
        corr = np.corrcoef(np.ndarray.flatten(ii),np.ndarray.flatten(jj))[0,1]
        vals.append(corr)
    return vals
        
def get_max_norm(a,b,c):
    vals = []
    for ii in [a,b,c]:
        vals.append(np.max(ii)/np.mean(ii))
    return vals

def get_mean_std(a,b,c):
    vals = []
    for ii in [a,b,c]:
        vals.append(np.mean(ii)/np.std(ii))
    return vals


In [182]:
path = '/Users/cwfink/Documents/School/UC_Berkeley/Classes/Spring_2018/AY250/HW/Fink_ay250_homework/hw_6/50_categories/'
path = '50_categories/'
X = [] 
Y = []

for subdir, dirs, files in os.walk(path):
    print(O)
    for file in files:
        if '.DS_Store' not in file:
            x, y = get_features(subdir+'/',file,lgc_return_label = True)
            X.append(x)
            Y.append(y)




['bat', 'bat', 'bat', 'bat']

In [63]:
1/50

0.02